<img align="center" src="figures/course.png" width="800">

#                                    16720 (B) Bag of Visual Words - Assignment 2

     Instructor: Kris Kitani                   TAs: Sheng-Yu, Jinkun, Rawal, Arka, Rohan

In [146]:
import nbimporter
import numpy as np
import scipy.ndimage
from skimage import io
import skimage.transform
import os,time
import util
import multiprocess
import threading
import queue
import torch
import torchvision
import torchvision.transforms

## For Autograding P4, ensure uploading `trained_conf_matrix.npy` and `trained_system_deep.npz`. 

## For extra credit, ensure uploading `trained_conf_matrix_vit.npy` and `trained_system_vit.npz`.

### Deep Learning Features - An Alternative to ``Bag of Words``

As we have discussed in class, another powerful method for scene classification in computer vision is the employment of convolutional neural networks (CNNs) - sometimes referred to generically as $deep learning$. It is important to understand how previously trained (pretrained) networks can be used as another form of feature extraction, and how they relate to classical Bag of Words (BoW) features. We will be covering details on how one chooses the network architecture and training procedures later in the course. For this question, however, we will be asking you to deal with the VGG-16 pretrained network. VGG-16 is a pretrained Convolutional Neural Network (CNN) that has been trained on approximately 1.2 million images from the ImageNet Dataset (``http://image-net.org/index``) by the Visual Geometry Group (VGG) at University of Oxford. The model can classify images into a 1000 object categories (e.g. keyboard, mouse, coffee mug, pencil).

One lesson we want you to take away from this exercise is to understand the effectiveness of $deep$ $features$ for general classification tasks within computer vision - even when those features have been previously trained on a different dataset (i.e. ImageNet) and task (i.e. object recognition). 

#### Extracting Deep Features

To complete this question, you need to install the ``torchvision`` library from Pytorch, a popular Python-based deep learning library.
If you are using the Anaconda package manager (``https://www.anaconda.com/download/``), this can be done with the following command:
```
            conda install pytorch torchvision -c pytorch
```
To check that you have installed it correctly, make sure that you can ``import torch`` in a Python interpreter without errors.
Please refer to ``https://pytorch.org/`` for more detailed installation instructions. 

#### Q4.1.1 (25 Points)

We want to extract out deep features corresponding to the convolutional layers of theVGG-16 network.  In this problem, we will use the trained weights from the VGG network directly, but implement our own operations. To load the network, use the line
```
        vgg16 = torchvision.models.vgg16(pretrained=True).double()
```
followed by ``vgg16.eval()``
The latter line ensures that the VGG-16 network is in evaluation mode, not training mode.

path_img = "./data/aquarium/sun_aztvjgubyrgvirup.jpg"

image = io.imread(path_img)

image = image.astype('float')/255

vgg16 = torchvision.models.vgg16(pretrained=True).double()

vgg16.eval()

We want you to complete a function that is able to output $VGG-16$ network outputs at the **fc7** layer in
```
    def extract_deep_feature(x,vgg16_weights):
```
where ``x`` refers to the input image and ``vgg16_weights`` is a structure containing the CNN's network parameters. In this function you will need to write sub-functions ``multichannel_conv2d``, ``relu``, ``max_pool2d``, and ``linear`` corresponding to the fundamental elements of the CNN: multi-channel convolution, rectified linear units (ReLU), max pooling, and fully-connected weighting.

We have provided a helper function ``util.get_VGG16_weights()`` that extracts the weight parameters of VGG-16 and its meta information. The returned variable is a numpy array of shape $L\times 3$, where $L$ is the number of layers in VGG-16. The first column of each row is a string indicating the layer type. The second/third columns may contain the learned weights and biases, or other meta-information (\eg kernel size of max-pooling). Please refer to the function docstring for details.

In order to build the ``extract_deep_feature`` function, you should run a for-loop through each layer index until layer **fc7**, which corresponds to **the second linear layer** (Refer to VGG structure to see where **fc7** is). **Remember**: the output of the preceding layer should be passed through as an input to the next.

Details on the sub-functions needed for the ``extract_deep_feature`` function can be found below.

Please use ``scipy.ndimage.convolve`` and ``numpy`` functions to implement these functions instead of using pytorch. Please keep speed in mind when implementing your function, for example, using double for loop over pixels is not a good idea.

``multichannel_conv2d(x,weight,bias)``:

a function that will perform multi-channel 2D convolution which can be defined as follows, 

\begin{equation}
\mathbf{y}^{(j)} = \sum_{k=1}^{K} \begin{bmatrix} \mathbf{x}^{(k)} * \mathbf{h}^{(j,k)} \end{bmatrix} + \mathbf{b}[j] 
\end{equation}

where $*$ denotes $2D$ convolution, $\mathbf{x} = \{ \mathbf{x}^{(k)} \}_{k=1}^{K}$ is our vectorized $K$-channel input signal, $\mathbf{h} = \{ \mathbf{h}^{(j,k)} \}_{k=1,j=1}^{K,J}$ is our $J \times K$ set of vectorized convolutional filters and $\mathbf{r} = \{ \mathbf{y}^{(j)} \}_{j=1}^{J}$ is our $J$ channel vectorized output response. Further, unlike traditional single-channel convolution CNNs often append a bias vector $\mathbf{b}$ whose $J$ elements are added to the $J$ channels of the output response. 

To implement ``multichannel_conv2d``, you can use the Scipy convolution function directly with for loops to cycle through the filters and channels (``scipy.ndimage.convolve()``). All the necessary details concerning the number of filters ($J$), number of channels ($K$), filter weights ($\mathbf{h}$) and biases ($\mathbf{b}$) can be inferred from the shapes/dimensions of the weights and biases. Notice that pytorch's convolution function actually does correlation, so to get similar answer as pytroch with scipy, you need to flip the kernel on both axes using ``np.flip()``.

In [184]:
def multichannel_conv2d(x,weight,bias):
    '''
    Performs multi-channel 2D convolution.

    [input]
    * x: numpy.ndarray of shape (H,W,input_dim)
    * weight: numpy.ndarray of shape (output_dim,input_dim,kernel_size,kernel_size)
    * weight is [JxK] where J is the number of filters, K is the number of features
    * bias: numpy.ndarray of shape (output_dim)

    [output]
    * feat: numpy.ndarray of shape (H,W,output_dim)
    '''
    h, w, input_dims = x.shape
    output_dims = weight.shape[0]

    '''
    HINTS:
    1.> for 2D convolution we need to use np.fliplr and np.flipud
    2.> can use scipy.ndimage.convolve with the flipped kernel
    3.> don't forget to add the bias
    '''
    
    channel_filts = []
    for img_channel in range(x.shape[2]):
        filt_out = []
        for filt_num, f in enumerate(weight):
            # Convolve the image channel with the filter
            filt_out.append(scipy.ndimage.convolve(x[:, :, img_channel], np.fliplr(np.flipud(f[img_channel]))))
        channel_filts.append(np.stack(filt_out))
    out = np.mean(np.stack(channel_filts), axis=0)
    
    return np.moveaxis(out, 0, -1) + bias

```relu(x)```:

a function that shall perform the Rectified Linear Unit (ReLU) which can be defined mathematically as,

\begin{equation}
\mbox{ReLU}(x) = \max_{x}(x, 0)
\end{equation}

and is applied independently to each element of the matrix/vector $x$ passed to it.

In [174]:
def relu(x):
    '''
    Rectified linear unit.

    [input]
    * x: numpy.ndarray

    [output]
    * y: numpy.ndarray
    '''
    x[x < 0] = 0
    return x


``max_pool2d(x,size):``

a function that shall perform max pooling over $x$ using a receptive field of size $size$ $\times$ $size$ (we assume a square receptive field here for simplicity).

  If the function receives a multi-channel input, then it should apply the max pooling operation across each input channel independently.
  
(Hint: making use of smart array indexing can drastically speed up the code.)

In [192]:
from math import ceil


def max_pool2d(x,size):
    '''
    2D max pooling operation.

    [input]
    * x: numpy.ndarray of shape (H,W,input_dim)
    * size: pooling receptive field

    [output]
    * y: numpy.ndarray of shape (H/size,W/size,input_dim)
    '''
    h, w, dims = x.shape
    '''
    HINTS:
    1.> estimate the shape you need to apply the pooling operation.
    2.> We can smart fill the padding with np.nan and then use np.nanmax to select the max (avoiding nan)
    3.> We can input the grid (start_x:end_x, start_y:end_y, dim) as smart array indexing to np.nanmax
    '''
    # The below code crashes the kernel probably because it takes up too much memory
    x_pad = np.zeros((x.shape[0]+size, x.shape[1]+size, x.shape[2]))
    for channel in range(x.shape[2]):
        x_pad[:, :, channel] = np.pad(x[:, :, channel], int(size/2), mode="constant", constant_values=np.nan)
    pool_h, pool_w = h // size, w // size
    out_pool = np.zeros((pool_h, pool_w, dims))
    
    for dim in range(dims):
        for i in range(pool_h):
            for j in range(pool_w):
                out_pool[i, j, dim] = np.nanmax(x_pad[2*i:2*(i+1), 2*j:2*(j+1), dim])
    
    return out_pool

``linear(x,W,b):``

a function that will compute a node vector where each element is a linear combination of the input nodes, written as

\begin{equation}
\mathbf{y}[j] = \sum_{k=1}^{K}\mathbf{W}[j,k] \mathbf{x}[k] + \mathbf{b}[j] 
\end{equation}

or more succinctly in vector form as $\mathbf{y} = \mathbf{W} \mathbf{x} + \mathbf{b}$ - where $\mathbf{x}$ is the $(K \times 1)$ input node vector, $\mathbf{W}$ is the $(J \times K)$ weight matrix, $\mathbf{b}$ is the $(J \times 1)$ bias vector and $\mathbf{y}$ is the~$(J \times 1)$ output node vector.

You should not need for-loops to implement this function.

In [176]:
def linear(x,W,b):
    '''
    Fully-connected layer.

    [input]
    * x: numpy.ndarray of shape (input_dim)
    * weight: numpy.ndarray of shape (output_dim,input_dim)
    * bias: numpy.ndarray of shape (output_dim)

    [output]
    * y: numpy.ndarray of shape (output_dim)
    '''

    y = np.matmul(W, x.flatten()) + b
    return y

You should ignore all ``DropoutLayer`` you encounter; they're functional only during the training phase.

VGG-16 assumes that all input imagery to the network is resized to $224 \times 224$ with the three color channels preserved (use ``skimage.transform.resize()`` to do this before passing any imagery to the network). And be sure to normalize the image using suggested mean and std before extracting the feature:
```
                                        mean=[0.485,0.456,0.406]}
                                        std=[0.229,0.224,0.225]}
```

In [178]:
from p4_1 import preprocess_image

For efficiency you should check that each sub-function is working properly before putting them all together - otherwise it will be hard to track any errors. To compare your implementation with pytroch, you should compare the extracted features between your ``extract_deep_feature``  and the pre-trained VGG-16 network.  ``evaluate_deep_extractor`` should come in handy in comparing the result of the two extracted features.

In [177]:
def extract_deep_feature(x, vgg16_weights):
    '''
    Extracts deep features from the given VGG-16 weights.

    [input]
    * x: numpy.ndarray of shape (H,W,3)
    * vgg16_weights: list of shape (L,3)

    [output]
    * feat: numpy.ndarray of shape (K)
    '''
    
    feat = np.copy(x)
    for num, layer in enumerate(vgg16_weights):
        print(f"Processing: {layer[0]}")
        if num == 33:
            break
        elif layer[0] == "conv2d":
            feat = multichannel_conv2d(feat, layer[1], layer[2])
        elif layer[0] == "relu":
            feat = relu(feat)
        elif layer[0] == "linear":
            feat = linear(feat, layer[1], layer[2])
        elif layer[0] == "maxpool2d":
            feat = max_pool2d(feat, layer[1])
        else:
            raise ValueError("Unknown layer type")
    return feat

In [179]:
def evaluate_deep_extractor(img, vgg16):
    '''
    Evaluates the deep feature extractor for a single image.

    [input]
    * image: numpy.ndarray of shape (H,W,3)
    * vgg16: prebuilt VGG-16 network.

    [output]
    * diff: difference between the two feature extractor's result
    '''
    img_torch = preprocess_image(img)

    vgg16_weights = util.get_VGG16_weights()
    feat = extract_deep_feature(np.transpose(img_torch.numpy(), (1,2,0)), vgg16_weights)
    
    with torch.no_grad():
        vgg_classifier = torch.nn.Sequential(*list(vgg16.classifier.children())[:-3])
        vgg_feat_feat = vgg16.features(img_torch[None, ])
        vgg_feat_feat = vgg_classifier(vgg_feat_feat.flatten())
    
    return np.sum(np.abs(vgg_feat_feat.numpy() - feat))

In [193]:
# NOTE: comment out the lines below before submitting to gradescope
# Visible test cases (for debugging)
path_img = "./data/aquarium/sun_aztvjgubyrgvirup.jpg"
image = io.imread(path_img)
image = image.astype('float') / 255

vgg16 = torchvision.models.vgg16(pretrained=True).double()
vgg16.eval()
error = evaluate_deep_extractor(image, vgg16)

# This error should be less than 1e-10
print("Error:", error)

Processing: conv2d
Processing: relu
Processing: conv2d
Processing: relu
Processing: maxpool2d


### Building a Visual Recognition System: Revisited

We want to compare how useful deep features are in a visual recognition system. Since the speed of the function ``` scipy.ndimage.convolve``` is not ideal, you can use the pytroch VGG-16 network directly (refer to the helper function ```evaluate_deep_extractor``` on how to use the pre-trained network as feature extractor).

#### Q4.2.1 (5 Points Autograder + WriteUp):
Implement the functions
```
                    def build_recognition_system(vgg16):
```
and
```
                    def eval_recognition_system(vgg16)}
```
both of which takes the pretrained VGG-16 network as the input arguments. 

The former function should produce ``trained_system_deep.npz`` as the output. 

Included will be:
* $features$ : a $N \times  K$ matrix containing all the deep features of the $N$ training images in the data set.
* $labels$ : an $N$ vector containing the labels of each of the images. ($features[i]$ will correspond to label $labels[i]$).

The latter function should produce the confusion matrix, as with the previous question.

Instead of using the histogram intersection similarity, write a function to just use the negative Euclidean distance (as larger values are more similar).

**Report the confusion matrix and accuracy for your results in your write-up. Can you comment in your writeup on whether the results are better or worse than classical BoW - why do you think that is?**

In [155]:
from p4_1 import get_image_feature

In [156]:
from p4_1 import build_recognition_system

In [157]:
from p4_1 import evaluate_recognition_system

In [158]:
### Run the code
# import nbimporter
# import torchvision
# from p4_1 import preprocess_image

# vgg16 = torchvision.models.vgg16(pretrained=True).double()
# vgg16.eval()

# #ret = build_recognition_system(vgg16, num_workers=4)
# conf_matrix, accuracy = evaluate_recognition_system(vgg16)
# # We expect the accuracy to be greater than 0.9
# print("Accuracy:", accuracy)
# # Visualize the confusion matrix
# def visualize_conf_matrix(conf_matrix):   
#     import seaborn as sns
#     import matplotlib.pyplot as plt
#     labels = ["AQRM", "PARK", "DSRT", "HGHWY", "KTCHN", "LNDROMT", "WFLL", "WNDMLL"]
#     ax = sns.heatmap(conf_matrix, annot=True, cmap='Blues')
#     ax.set_title('Seaborn Confusion Matrix with labels\n\n')
#     ax.set_xlabel('\nPredicted Values')
#     ax.set_ylabel('Actual Values ')
#     ax.xaxis.set_ticklabels(labels)
#     ax.yaxis.set_ticklabels(labels)
#     plt.show()

# print("My Confusion Matrix:")
# visualize_conf_matrix(conf_matrix)

### (Extra Credit) Extract ViT features

**q4.3.1 (10 points)**
For extra credit, we ask you to try out the features from a recently popular architecture -- Vision Transformers (ViTs).

You'll learn more about ViT later in the course. As an overview, instead of convolutions, ViT treat image patches as tokens, and process the tokens via self-attention. For more details, feel free to check out the paper "An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale" (``https://arxiv.org/abs/2010.11929``).

The goal in this problem is not to understand ViT. We expect you to learn how to extract features from an unseen architecture (e.g., a new model released straight out of arxiv). Here you'll learn how to:
1. Iterate through the pytorch nn modules without digging too much into the source code.
2. Learn another way to extract features -- `register_forward_hook()`

In this problem, we'll be using an ImageNet-pretrained ViT from `pytorch_pretrained_vit`. In case you haven't installed it, run:

```pip install pytorch_pretrained_vit```

Below is a script that loads a pre-trained ViT and prints out all the submodules.

In [159]:
from pytorch_pretrained_vit import ViT


# loads a pretrained ViT
vit_model = ViT('B_16_imagenet1k', pretrained=True)
vit_model.eval()

# iterate through the submodules and print out names
for name, module in vit_model.named_modules():
    print(name)

KeyboardInterrupt: 

`named_modules()` will recursively find out all the `nn.Modules` defined inside `vit_model`. For example: 
1. `vit_model` has children `patch_embedding`, `positional_embedding`, `transformer`, `norm`, `fc`.
2. Inside the `transformer` module, `transformer` has a child `transformer.blocks`.
3. `transformer.blocks` module has children `transformer.blocks.0`, `transformer.blocks.1`, etc.
4. The logic applies recursively.

Unfortunately, calling just `named_modules()` will not give you the information about the ordering each module is used.

However, if you have a basic understanding of the architecture code, you can extract features quickly, regardless of how complicated the `forward()` function is. In ViT, we know that the `patch_embedding` and `positional_embedding` is passed into `transformer`, and the output of `transformer` is then passed into `norm` then `fc`.

Here, we want to extract features from the output of the `transformer` module. We will use `register_forward_hook()` to do this. Please read the documentation [here](https://pytorch.org/docs/stable/generated/torch.nn.Module.html?highlight=forward%20hook#torch.nn.Module.register_forward_hook) for details. Essentially, we want to "hook" a function that records the output of the `transformer` module, whenever `vit_model` is called. We'll define a class FeatureExtractor to do this.

**Note:** The input image size for ViT will be 384x384, and normalization uses a different mean and std:
```
                                        mean=[0.5, 0.5, 0.5]
                                        std=[0.5, 0.5, 0.5]
```

In [ ]:
from p4_1 import preprocess_image

class FeatureExtractor:
    '''
    A class that takes in a nn.Module model and extracts feature from specified layer name.
    '''
    def __init__(self, model, layername='transformer'):
        self.extracted_feature = None
        self.model = model              # This will be vit_model in our case
        self.layername = layername

        # Apply hook to the transformer module
        '''
        HINTS:
        1.> The for loop of named_modules() we provided will be useful here.
        2.> Apply feature_extract_hook() to the transformer module using register_forward_hook()
        '''
        for name, module in model.named_modules():
            # Register the forward hook
            if name == layername:
                module.blocks[-1].norm2.register_forward_hook(self.feature_extract_hook)

    def feature_extract_hook(self, module, input, output):
        '''
        A function hook that extracts the module's output to the global variable `extracted_feature`

        [input]
        * module: module of interest
        * input: input of the module
        * output: output of the module
        '''

        '''
        HINTS:
        1.> You don't need to use all the arguments in this function.
        2.> What you need to do in this function should be really simple.
        '''
        self.extracted_feature = output.detach()

    def extract_feature(self, img):
        '''
        Takes in an image, feed it to the model, and outputs the desired feature.
        
        [input]
        * x: preprocessed image
        
        [output]
        * feature: feature extracted from the specified layer name
        '''
        x = preprocess_image_vit(img).float()
        
        # simply run a forward pass of the model
        with torch.no_grad():
            self.model.forward(x.unsqueeze(0))

        # feature will be extracted in self.extracted_feature already, thanks to the hook
        # you might wonder why we take only part of the output as feature
        # this is because we are only using the "class token" as the feature
        # for more details, please read the paper!
        return self.extracted_feature.numpy()[0, 0]


def preprocess_image_vit(image):
    '''
    Preprocesses the image to load into the prebuilt network.

    [input]
    * image: numpy.ndarray of shape (H,W,3)

    [output]
    * image_processed: torch.array of shape (3,H,W)
    '''
    return preprocess_image(image, size = (384, 384, 3), mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])

**q4.3.2 (5 points + Write up)** Now we can extract features from ViT to build our recognition system. The following code will be mostly the same as the one you did before.

Write up: Report the confusion matrix and accuracy for your results in your write-up. Can you comment in your writeup on whether the results are better or worse than VGG - why do you think that is? A short answer is okay.

In [ ]:
def build_recognition_system_vit(vit_feat_extractor):
    '''
    Creates a trained recognition system by generating training features from all training images.

    [input]
    * vit_feat_extractor: feature extractor for ViT

    [saved]
    * features: numpy.ndarray of shape (N,K)
    * labels: numpy.ndarray of shape (N)
    '''

    train_data = np.load("./data/train_data.npz", allow_pickle=True)
    args = []

    # Parse out the training files
    train_files = train_data.get("files")
    if train_files is None:
        raise ValueError("No valid training files available :(")
    train_files = ["./data/" + str(file) for file in train_files]

    # Do the processing
    out  = np.zeros((len(train_files), 768))
    for idx, train_file in enumerate(train_files):
        out[idx] = vit_feat_extractor.extract_feature(skimage.io.imread(train_file))
    
    '''
    HINTS:
    1.> reorder the features to their correct place as input
    '''
    print("done", out.shape)
    labels = train_data.get("labels")
    np.savez('trained_system_vit.npz', features=out, labels=labels)

In [ ]:
import nbimporter
from p3 import distance_to_set
import numpy as np

def evaluate_recognition_system_vit(vit_feat_extractor):
    '''
    Evaluates the recognition system for all test images and returns the confusion matrix.

    [input]
    * vit_feat_extractor: feature extractor for ViT

    [output]
    * conf: numpy.ndarray of shape (8,8)
    * accuracy: accuracy of the evaluated system
    '''

    test_data = np.load("./data/test_data.npz")
    trained_system = np.load("trained_system_vit.npz")
    
    image_names = test_data['files']
    image_names = ["./data/" + str(file) for file in image_names]
    test_labels = test_data['labels']

    trained_features = trained_system['features']
    train_labels = trained_system['labels']
    print("Trained features shape: ", trained_features.shape)
    
    # Gather arguments for multiprocessing
    args = []
    for idx, img in enumerate(image_names):
        args.append((idx, img))

    # Dictionary to hold on to our predictions
    pred_labels = []

    for arg in args:
        # Extract the feature for the test image
        test_features = vit_feat_extractor.extract_feature(skimage.io.imread(arg[1]))

        # Calculate the distance between the two features
        dist = np.sum((test_features - trained_features) ** 2, axis = 1)
        pred_label = train_labels[np.argmin(dist)]
        pred_labels.append(pred_label)

    pred_labels = np.array(pred_labels)
    print("Predicted labels shape: ", pred_labels.shape)

    # Compute the confusion matrix
    num_labels  = len(np.unique(test_labels))
    conf_matrix = np.zeros((num_labels, num_labels))

    for i in range(len(test_labels)):
        conf_matrix[test_labels[i], pred_labels[i]] += 1

    # Calculate the accuracy
    accuracy = np.trace(conf_matrix) / len(image_names)

    # Visualize the confusion matrix
    def visualize_conf_matrix(conf_matrix):   
        import seaborn as sns
        import matplotlib.pyplot as plt
        labels = ["AQRM", "PARK", "DSRT", "HGHWY", "KTCHN", "LNDROMT", "WFLL", "WNDMLL"]
        ax = sns.heatmap(conf_matrix, annot=True, cmap='Blues')
        ax.set_title('Seaborn Confusion Matrix with labels\n\n')
        ax.set_xlabel('\nPredicted Values')
        ax.set_ylabel('Actual Values ')
        ax.xaxis.set_ticklabels(labels)
        ax.yaxis.set_ticklabels(labels)
        plt.show()

    print("My Confusion Matrix:")
    visualize_conf_matrix(conf_matrix)

    # Save confusion matrix to a file
    np.save("./conf_matrix_vit.npy", conf_matrix)
    return conf_matrix, accuracy

In [ ]:
# NOTE: comment out the lines below before submitting to gradescope
### Run the code
vit = ViT('B_16_imagenet1k', pretrained=True)
vit.eval()
vit_feat_extractor = FeatureExtractor(vit)
# build_recognition_system_vit(vit_feat_extractor)
conf_matrix, accuracy = evaluate_recognition_system_vit(vit_feat_extractor)
print("Accuracy:", accuracy)

Loaded pretrained weights.
Trained features shape:  (1000, 768)


KeyboardInterrupt: 